# Using TVM Complier on CPU

![flow](https://raw.githubusercontent.com/apache/tvm-site/main/images/tutorial/overview.png)

## Import dependencies

In [1]:
import tvm 
import numpy as np
import onnx
import os
import tvm.relay as relay
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import warnings
import time
warnings.filterwarnings('ignore')

/home/khadas/.pyenv/versions/3.7.12/lib/python3.7/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/home/khadas/.pyenv/versions/3.7.12/lib/python3.7/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/home/khadas/.pyenv/versions/3.7.12/lib/python3.7/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/home/khadas/.pyenv/versions/3.7.12/lib/python3.7/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/home/khadas/.pyenv/versions/3.7.12/lib/python3.7/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/home/khadas/.pyenv/versions/3.7.12/lib/python3.7/

## Prepare our own dataset

In [2]:
# Actions that we try to detect 5 classes
actions = np.array(['hello', 'iloveyou', 'yes','donothing'])
# Map actions to integers
label_map = {label:num for num, label in enumerate(actions)}
print(f'Label map: {label_map}')

# Map x, y for data and labels
DATA_PATH = os.path.join('MP_Data') 
no_sequences = 100
sequence_length = 30
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

# One hot encoding
y = to_categorical(labels).astype(int)
print(f'y shape: {y.shape}')
# print(f'Y: \n{y}')
X = np.array(sequences)
print(f'X shape: {X.shape}')

# Train Test split with testing size 0.1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=11)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')



Label map: {'hello': 0, 'iloveyou': 1, 'yes': 2, 'donothing': 3}
y shape: (400, 4)
X shape: (400, 30, 1662)
X_train shape: (360, 30, 1662)
X_test shape: (40, 30, 1662)
y_train shape: (360, 4)
y_test shape: (40, 4)


## Load TF Onnx model

In [3]:
# Model Path
model_path = 'slModel.onnx'
# Load onnx model
onnx_model = onnx.load(model_path)

# Set random seed for random numpy
np.random.seed(3)

### Show Onnx model input

In [4]:
print(f'Model inputs: {onnx_model.graph.input}')

Model inputs: [name: "lstm_10_input"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "unk__303"
      }
      dim {
        dim_value: 30
      }
      dim {
        dim_value: 1662
      }
    }
  }
}
]


### Show Onnx model output

In [5]:
print(f'Model output: {onnx_model.graph.output}')

Model output: [name: "dense_9"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "unk__304"
      }
      dim {
        dim_value: 4
      }
    }
  }
}
]


## Convert Onnx to Relay for compilation

In [6]:
# compiler target
target = 'llvm'

# Input parameter name
input_name = onnx_model.graph.input[0].name
# Input shape of the model
input_shape = (1, 30, 1662)
# Tell the ralay which ONNX parameter is input
shape_dict = {input_name: input_shape}
print(shape_dict)
# Passing information into relay which is from onnx model
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

{'lstm_10_input': (1, 30, 1662)}


### Build Relay

In [7]:
# The optimization level of this pass.
opt_level = 3
# Each pass context contains a number of auxiliary information that is used to help an optimization pass.
with tvm.transform.PassContext(opt_level=opt_level):
    # builds a Relay function to run on TVM graph executor.
    lib = relay.build(mod, target=target, params=params)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


### Create a runtime module

In [8]:
# device to optimize to (CPU)
dev = tvm.device(str(target), 0)

# Create a runtime executor module given a graph and module.
# Wrapper runtime module.
module = tvm.contrib.graph_executor.GraphModule(lib["default"](dev))

## Benchmark module performance

`benchmark(device, func_name='run', repeat=5, number=5, min_repeat_ms=None, limit_zero_time_iterations=100, end_to_end=False, cooldown_interval_ms=0, repeats_to_cooldown=1, **kwargs)¶`

Calculate runtime of a function by repeatedly calling it.

Use this function to get an accurate measurement of the runtime of a function. The function is run multiple times in order to account for variability in measurements, processor speed or other external factors. Mean, median, standard deviation, min and max runtime are all reported. On GPUs, CUDA and ROCm specifically, special on-device timers are used so that synchonization and data transfer operations are not counted towards the runtime. This allows for fair comparison of runtimes across different functions and models. The end_to_end flag switches this behavior to include data transfer operations in the runtime.

Returns
timing_results – Runtimes of the function. Use .mean to access the mean runtime, use .results to access the individual runtimes (in seconds).

Return type
BenchmarkResult

### Before optimization

In [9]:
timing_results = module.benchmark(device = dev, repeat = 30, number = 1)
print(timing_results)

Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  16.0726      15.1003      28.0091      14.6035       2.8954   
               


In [10]:
print(f'Mean time before optimization : {timing_results.mean}')

Mean time before optimization : 0.01607257056666667


## Tune the model

The previous model was compiled to work on the TVM runtime, but did not include any platform specific optimization. In this section, we will show you how to build an optimized model using TVM to target your working platform.

In some cases, we might not get the expected performance when running inferences using our compiled module. In cases like this, we can make use of the auto-tuner, to find a better configuration for our model and get a boost in performance. Tuning in TVM refers to the process by which a model is optimized to run faster on a given target. This differs from training or fine-tuning in that it does not affect the accuracy of the model, but only the runtime performance. As part of the tuning process, TVM will try running many different operator implementation variants to see which perform best. The results of these runs are stored in a tuning records file.

In the simplest form, tuning requires you to provide three things:

- the target specification of the device you intend to run this model on

- the path to an output file in which the tuning records will be stored

- a path to the model to be tuned.

### Import model tuning dependencies

In [8]:
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner
from tvm import autotvm

### Set a LocalRunner

Set up some basic parameters for the runner. The runner takes compiled code
that is generated with a specific set of parameters and measures the
performance of it. ``number`` specifies the number of different
configurations that we will test, while ``repeat`` specifies how many
measurements we will take of each configuration. ``min_repeat_ms`` is a value
that specifies how long need to run configuration test. If the number of
repeats falls under this time, it will be increased. This option is necessary
for accurate tuning on GPUs, and is not required for CPU tuning. Setting this
value to 0 disables it. The ``timeout`` places an upper limit on how long to
run training code for each tested configuration.




In [9]:
number = 1
repeat = 30
min_repeat_ms = 0  # since we're tuning on a CPU, can be set to 0
timeout = 10  # in seconds

# create a TVM runner on local device
runner = autotvm.LocalRunner(
    number=number,
    repeat=repeat,
    timeout=timeout,
    min_repeat_ms=min_repeat_ms,
    enable_cpu_cache_flush=True,
)

### Tuning options

Simple Structure for holding tuning options.

In [10]:
# Tuning parameters
tuning_option = {
    "tuning_records": "signLanguageModel-v3-autotuning-cpu.json",
    "use_transfer_learning": True,
    "tuner": "xgb",
    "n_trial": 1500,
    "early_stopping": 100,
    "measure_option": autotvm.measure_option(
        builder=autotvm.LocalBuilder(build_func="default"), runner=runner
    ),
}

### Extract Task from the model

Task is a tunable composition of template functions.

Tuner takes a tunable task and optimizes the joint configuration space of all the template functions in the task. This module defines the task data structure, as well as a collection(zoo) of typical tasks of interest.

Definition of task function.

Task can be constructed from tuple of func, args, and kwargs. func is a state-less function, or a string that registers the standard task.



In [14]:
# begin by extracting the tasks from the onnx model
tasks = autotvm.task.extract_from_program(mod["main"], target=target, params=params)

[19:10:26] /home/khadas/tvm/src/runtime/threading_backend.cc:343: Warning: more than two frequencies detected! Forced big_count_ to 4


### Tune Task sequentially

In [15]:
# Create tuning function
def tune_tasks(
    tasks,
    measure_option,
    tuner="xgb",
    n_trial=1000,
    early_stopping=None,
    tuning_records="tuning.json",
    use_transfer_learning=True,
):
    # create tmp log file
    tmp_log_file = tuning_records + ".tmp"
    if os.path.exists(tmp_log_file):
        os.remove(tmp_log_file)

    for i, tsk in enumerate(reversed(tasks)):
        prefix = "[Task %2d/%2d] " % (i + 1, len(tasks))

        # create tuner
        if tuner == "xgb" or tuner == "xgb-rank":
            tuner_obj = XGBTuner(tsk, loss_type="rank")
        elif tuner == "xgb_knob":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="knob")
        elif tuner == "xgb_itervar":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="itervar")
        elif tuner == "xgb_curve":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="curve")
        elif tuner == "ga":
            tuner_obj = GATuner(tsk, pop_size=50)
        elif tuner == "random":
            tuner_obj = RandomTuner(tsk)
        elif tuner == "gridsearch":
            tuner_obj = GridSearchTuner(tsk)
        else:
            raise ValueError("Invalid tuner: " + tuner)

        if use_transfer_learning:
            if os.path.isfile(tmp_log_file):
                tuner_obj.load_history(autotvm.record.load_from_file(tmp_log_file))

        # process tuning
        tsk_trial = n_trial
        tuner_obj.tune(
            n_trial=tsk_trial,
            early_stopping=early_stopping,
            measure_option=measure_option,
            callbacks=[
                autotvm.callback.progress_bar(tsk_trial, prefix=prefix),
                autotvm.callback.log_to_file(tmp_log_file),
            ],
        )

    # pick best records to a cache file
    autotvm.record.pick_best(tmp_log_file, tuning_records)
    os.remove(tmp_log_file)

### Start tuning

In [16]:
tune_tasks(tasks, **tuning_option)

[Task  1/16]  Current/Best:    0.00/   0.00 GFLOPS | Progress: (0/1500) | 0.00 s

[19:12:23] /home/khadas/tvm/src/runtime/threading_backend.cc:343: Warning: more than two frequencies detected! Forced big_count_ to 4
[19:12:23] /home/khadas/tvm/src/runtime/threading_backend.cc:343: Warning: more than two frequencies detected! Forced big_count_ to 4
[19:12:25] /home/khadas/tvm/src/runtime/threading_backend.cc:343: Warning: more than two frequencies detected! Forced big_count_ to 4
[19:12:26] /home/khadas/tvm/src/runtime/threading_backend.cc:343: Warning: more than two frequencies detected! Forced big_count_ to 4


[Task  1/16]  Current/Best:    0.14/   0.79 GFLOPS | Progress: (8/1500) | 14.17 s

[19:12:30] /home/khadas/tvm/src/runtime/threading_backend.cc:343: Warning: more than two frequencies detected! Forced big_count_ to 4
[19:12:32] /home/khadas/tvm/src/runtime/threading_backend.cc:343: Warning: more than two frequencies detected! Forced big_count_ to 4
[19:12:35] /home/khadas/tvm/src/runtime/threading_backend.cc:343: Warning: more than two frequencies detected! Forced big_count_ to 4
[19:12:39] /home/khadas/tvm/src/runtime/threading_backend.cc:343: Warning: more than two frequencies detected! Forced big_count_ to 4
[19:12:39] /home/khadas/tvm/src/runtime/threading_backend.cc:343: Warning: more than two frequencies detected! Forced big_count_ to 4


[Task  1/16]  Current/Best:    0.20/   1.23 GFLOPS | Progress: (16/1500) | 23.19 s

...

[Task 16/16]  Current/Best:    0.79/   8.16 GFLOPS | Progress: (32/1500) | 55.43 s

[19:36:33] /home/khadas/tvm/src/runtime/threading_backend.cc:343: Warning: more than two frequencies detected! Forced big_count_ to 4


[Task 16/16]  Current/Best:    1.87/   8.16 GFLOPS | Progress: (36/1500) | 67.58 s

[19:36:45] /home/khadas/tvm/src/runtime/threading_backend.cc:343: Warning: more than two frequencies detected! Forced big_count_ to 4


 Done.


### Compiling and Optimized Model with Tuning Data

The tuning records stored in `signLanguageModel-v1-autotuning-cpu.json`. 

The compiler will use the results to generate high performance code for the model on your specified target.

Now that tuning data for the model has been collected, we can re-compile the model using optimized operators to speed up the computations.

In [11]:
# Recompile the model from the record
with autotvm.apply_history_best(tuning_option["tuning_records"]):
    with tvm.transform.PassContext(opt_level=3, config={}):
        lib = relay.build(mod, target=target, params=params)

# device CPU
dev = tvm.device(str(target), 0)
module = tvm.contrib.graph_executor.GraphModule(lib["default"](dev))

print('Optimized module loaded!!!')


Optimized module loaded!!!


## Comparing the Tuned and Untuned models

In [18]:
optimized_time = module.benchmark(device = dev, repeat = 30, number = 1)

print(f'Unoptimized : {timing_results}')

print(f'Optimized : {optimized_time}')


Unoptimized : Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  16.0726      15.1003      28.0091      14.6035       2.8954   
               
Optimized : Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  12.9731      12.6457      14.6032      12.4026       0.6231   
               


## Test the model performance

### Check output shape from the model

In [12]:
sign_test_data = np.expand_dims(X_test[22], axis=0)
sign_test_data.shape

(1, 30, 1662)

In [13]:
sign_actual_data = y_test[22]
sign_actual_data

array([1, 0, 0, 0])

In [14]:
# Output result
dtype = 'float32'
# set input to the module
module.set_input(input_name, sign_test_data)
# run forward execution
module.run()

tvm_output = module.get_output(0).numpy()
print(f'TVM output shape: {tvm_output.shape}')
print(f'TVM Output : {tvm_output}')
print(f'Output: {actions[np.argmax(tvm_output[0])]}')
print(f'Actual test data output: {actions[np.argmax(sign_actual_data)]}')

TVM output shape: (1, 4)
TVM Output : [[9.6187323e-01 3.7256207e-02 3.1616242e-04 5.5441313e-04]]
Output: hello
Actual test data output: hello


In [15]:
# Import 
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import mediapipe as mp
import time

# Define API model to use as a MP hoslistic
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

# Functions
def mediapipe_detection(image, model):
    '''Detect the landmarks of the image using the mediapipe model'''
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    '''Function to draw landmarks from the detected image'''
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    '''Same as draw_landmarks but with different colors and thickness'''
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    
def extract_keypoints(results):
    '''Extract keypoints from the results and combine them into a single array'''
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

colors = [(245,117,16), (117,245,16), (16,117,245), (116,57,39)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

# Real time prediction
sequence = []
sentence = []
threshold = 0.7
count = 0
prev_frame_time = 0
new_frame_time = 0
cap = cv2.VideoCapture(33)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections with mediapipe
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic with my model
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        count += 1
        sequence = sequence[-30:]

        # calculate frame rate
        new_frame_time = time.time()
        # fps will be number of frame processed in given time frame
        fps = 1/(new_frame_time - prev_frame_time)
        prev_frame_time = new_frame_time
        # convert into int
        fps = int(fps)
        # convert into string
        fps = str(fps)
        cv2.putText(image, f'FPS: {fps}', (300, 100), cv2.FONT_HERSHEY_COMPLEX, 2, (100, 255, 0), 2, cv2.LINE_AA)

        # Start prediction when the sequence equal to 30 frames + time for transition about 5 frames
        if len(sequence) == 30:
            # Inference
            # Output result
            dtype = 'float32'
            # set input to the module
            module.set_input(input_name, np.expand_dims(sequence, axis = 0))
            # run forward execution
            module.run()
            # get prediction result
            res = module.get_output(0).numpy()[0]


            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            count = 0
            # print(actions[np.argmax(res)])
            
            
            #3. Visualize logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()



INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
